In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
import re

In [50]:
from tqdm.auto import tqdm
from scipy.sparse import csr_matrix, save_npz, load_npz

In [41]:
import pickle

In [4]:
%load_ext autoreload
%autoreload 2
from project_transformer import Data_Wrangling

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
warnings.filterwarnings("ignore")

# Load Some Data to preprocess Vietnamese languages

In [5]:
# EMOJI
with open('../DATA/files/emojicon.txt', 'r', encoding="utf8") as file:
    emoji_lst = file.read().split('\n')
    emoji_dict = {}
    for line in emoji_lst:
        key, value = line.split('\t')
        emoji_dict[key] = str(value)

# TEEN CODE
with open('../DATA/files/teencode.txt', 'r', encoding="utf8") as file:
    teen_lst = file.read().split('\n')
    teen_dict = {}
    for line in teen_lst:
        key, value = line.split('\t')
        teen_dict[key] = str(value)

# ENG VIET
with open('../DATA/files/english-vnmese.txt', 'r', encoding="utf8") as file:
    eng_lst = file.read().split('\n')
    eng_dict = {}
    for line in eng_lst:
        key, value = line.split('\t')
        eng_dict[key] = str(value)

# WRONG WORD
with open('../DATA/files/wrong-word.txt', 'r', encoding="utf8") as file:
    wrong_lst = file.read().split('\n')

# STOP WORD
with open('../DATA/files/vietnamese-stopwords.txt', 'r', encoding="utf8") as file:
    stop_lst = file.read().split('\n')

# Load file data:

In [6]:
use_cols = ['product_id','sub_category','price','rating','clean_desc','product_name','clean_prd_name']
clean_prd_df = pd.read_csv('../DATA/clean_products.csv',usecols=use_cols)

In [7]:
clean_prd_df.head()

,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


## Check na values from clean data

In [8]:
drop_idx = clean_prd_df[clean_prd_df['clean_prd_name'].apply(lambda x: isinstance(x,float))].index

In [9]:
clean_prd_df = clean_prd_df.drop(index=drop_idx)

In [10]:
clean_prd_df = clean_prd_df.reset_index(drop=True)

# Cosine_similarity
- link https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
- link https://en.wikipedia.org/wiki/Cosine_similarity

In [131]:
tf_model = TfidfVectorizer(analyzer='word', stop_words=stop_lst,max_features=None)
tf_model.fit(raw_documents=clean_prd_df['clean_prd_name'])

TfidfVectorizer(stop_words=['Danh Mục', 'Shopee', 'Thời Trang Nam', 'a_lô',
                            'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô',
                            'amen', 'anh', 'anh_ấy', 'ba', 'ba_ba', 'ba_bản',
                            'ba_cùng', 'ba_họ', 'ba_ngày', 'ba_ngôi', 'ba_tăng',
                            'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bao_nả',
                            'bay_biến', 'biết', 'biết_bao', 'biết_bao_nhiêu',
                            'biết_chắc', ...])

In [133]:
#tf_model.vocabulary_ 

In [134]:
tfidf_matrix = tf_model.transform(raw_documents=clean_prd_df['clean_prd_name'])

In [135]:
type(tfidf_matrix)

scipy.sparse._csr.csr_matrix

In [137]:
#tfidf_matrix[0]

In [37]:
# cosine_similarities = cosine_similarity(tfidf_matrix, 
#                                         tfidf_matrix,
#                                         dense_output=True)

In [138]:
#tfidf_matrix[0]

In [91]:
type(cosine_similarities)

numpy.ndarray

In [92]:
cosine_similarities[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [98]:
# với mỗi sản phẩm, lấy 3 sản phẩm tương quan nhất
# results = {}

# for idx, row in tqdm(clean_prd_df.iterrows(),'Calcualte the high correaltion',total=clean_prd_df.shape[0]):
#     similar_indices = cosine_similarities[idx].argsort()[-6:-1]
#     similar_items = [(cosine_similarities[idx][i]) for i in similar_indices]
#     similar_items = [(cosine_similarities[idx][i], clean_prd_df.index[i]) for i in similar_indices]
#     #print(similar_items[0:])
#     results[idx] = similar_items[0:]

In [99]:
#results[0]

## Save file

In [100]:
#np.save('../DATA/cosine_sim_prd.npy', cosine_similarities)

In [139]:
# Save the TF-IDF vectorizer using pickle
with open('../DATA/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tf_model, f)

In [140]:
# Save the sparse matrix
save_npz('../DATA/tfidf_matrix.npz', tfidf_matrix)

## Apply Cosine Similarity

### Load model and matrix:

In [141]:
with open('../DATA/tfidf_vectorizer.pkl', 'rb') as f:
    loaded_tfidf_model = pickle.load(f)
type(loaded_tfidf_model)

sklearn.feature_extraction.text.TfidfVectorizer

In [142]:
load_tfidf_matrix = load_npz('../DATA/tfidf_matrix.npz')
type(load_tfidf_matrix)

scipy.sparse._csr.csr_matrix

### Build Function

In [143]:
final_df = clean_prd_df.reset_index().rename(columns={'index':'item_id'})
print("Shape of final DF:", final_df.shape)
final_df.head()

Shape of final DF: (49594, 8)


,item_id,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


In [183]:
final_df.to_csv(path_or_buf='../DATA/final_details.csv',index=False)

In [144]:
text_ = final_df.loc[final_df['item_id'] == 10,'clean_prd_name'].values[0]
text_

'áo lỗ nam cao_cấp aristino thun lỗ_lót sơ_mi màu trắng trơn đơn_giản_dáng classic_chất cotton mềm'

In [145]:
txt_sparse_matrix = loaded_tfidf_model.transform([text_])
type(txt_sparse_matrix)

scipy.sparse._csr.csr_matrix

In [146]:
cosine_text_ = cosine_similarity(X=txt_sparse_matrix,Y=load_tfidf_matrix)
cosine_text_

array([[0.1307439 , 0.17763791, 0.14192402, ..., 0.        , 0.01230257,
        0.02827438]])

In [169]:
top5_prd = cosine_text_.flatten().argsort()[-6:-1]
top5_score = [(cosine_text_.flatten()[i]) for i in top5_indices]

# Extract Product
ls_item = []
for id, sim in zip(top5_prd,top5_score):
    sim_item = {'ite_id': id, 'score':sim}
    ls_item.append(sim_item)

In [170]:
ls_item

[{'ite_id': 1039, 'score': 0.3501156647875863},
 {'ite_id': 40962, 'score': 0.3569730458639763},
 {'ite_id': 795, 'score': 0.3844639807598642},
 {'ite_id': 1046, 'score': 0.4157450865077967},
 {'ite_id': 121, 'score': 0.6654457923085177}]

In [173]:
# Sample text
text_ = final_df.loc[final_df['item_id'] == 10,'clean_prd_name'].values[0]

# Get Sparse Matrix from TFIDF
txt_sparse_matrix = loaded_tfidf_model.transform([text_])

# Calcualte the cosine score
cosine_text_ = cosine_similarity(X=txt_sparse_matrix,Y=load_tfidf_matrix)

top5_prd = cosine_text_.flatten().argsort()[-6:-1]
top5_score = [(cosine_text_.flatten()[i]) for i in top5_indices]

# Extract Product
ls_item = []
for id, sim in zip(top5_prd,top5_score):
    sim_item = {'item_id': id, 'score':sim}
    ls_item.append(sim_item)

ls_item

[{'item_id': 1039, 'score': 0.3501156647875863},
 {'item_id': 40962, 'score': 0.3569730458639763},
 {'item_id': 795, 'score': 0.3844639807598642},
 {'item_id': 1046, 'score': 0.4157450865077967},
 {'item_id': 121, 'score': 0.6654457923085177}]

## Function

In [180]:
def recommend_cosine(text_,
                     df,
                     tf_idf_model,
                     tf_idf_arr,
                     num = 5):
    # Get Sparse Matrix from TFIDF
    txt_sparse_matrix = loaded_tfidf_model.transform([text_])
    
    # Calcualte the cosine score
    cosine_text_ = cosine_similarity(X=txt_sparse_matrix,
                                     Y=load_tfidf_matrix)
    
    top5_prd = cosine_text_.flatten().argsort()[-num:-1]
    top5_score = [(cosine_text_.flatten()[i]) for i in top5_indices]
    
    # Extract Product
    ls_item = []
    for id, sim in zip(top5_prd,top5_score):
        sim_item = {'item_id': id, 'score':sim}
        ls_item.append(sim_item)
    print(f"San Pham user nhap: {text_}\n\n")
    for rec in ls_item:
        #print(rec[1])
        print(f"Recommnended:\tItem ID: {rec['item_id']}, {df.loc[df['item_id'] == rec['item_id'],'product_name'].values[0]}, (score: {rec['score']})\n")
    get_item =  df['item_id'].isin(top5_prd)
    
    return df[get_item]

In [181]:
text_ = final_df.loc[final_df['item_id'] == 10,'clean_prd_name'].values[0]
#print(text_)
recm_df = recommend_cosine(text_=text_,df=final_df,tf_idf_model=loaded_tfidf_model,tf_idf_arr=load_tfidf_matrix)

San Pham user nhap: áo lỗ nam cao_cấp aristino thun lỗ_lót sơ_mi màu trắng trơn đơn_giản_dáng classic_chất cotton mềm


Recommnended:	Item ID: 40962, Áo sơ mi Cotton dài tay trơn màu dành cho nam, (score: 0.3501156647875863)

Recommnended:	Item ID: 795, Áo Ba Lỗ Nam ARISTINO Chất Cotton Basic ATTR05 - AC21, (score: 0.3569730458639763)

Recommnended:	Item ID: 1046, Áo Ba Lỗ Nam ARISTINO - ATTR05 Cotton Siêu Mát, (score: 0.3844639807598642)

Recommnended:	Item ID: 121, Áo Ba Lỗ Nam Cao Cấp RAMSEY Thun 3 Lỗ Lót Trong Sơ Mi Màu Trắng Trơn Dáng Classic Chất Cotton Mềm Mịn RSC01, (score: 0.4157450865077967)



In [182]:
recm_df

,item_id,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
121,121,19121,Áo Ba Lỗ Nam Cao Cấp RAMSEY Thun 3 Lỗ Lót Tron...,Áo Ba Lỗ,29000.0,5.0,áo lỗ thương_hiệu ramsey chiều áo không_thể ta...,áo lỗ nam cao_cấp ramsey thun lỗ_lót sơ_mi màu...
795,795,19296,Áo Ba Lỗ Nam ARISTINO Chất Cotton Basic ATTR05...,Áo Ba Lỗ,124375.0,5.0,áo lỗ thương_hiệu aristino chất_liệu hàng gửi ...,áo lỗ nam aristino chất cotton
1046,1046,19547,Áo Ba Lỗ Nam ARISTINO - ATTR05 Cotton Siêu Mát,Áo Ba Lỗ,111000.0,5.0,áo lỗ thương_hiệu aristino chiều áo không_thể ...,áo lỗ nam aristino cotton
40962,40962,161415,Áo sơ mi Cotton dài tay trơn màu dành cho nam,Trang Phục Truyền Thống,470000.0,0.0,phục truyền_thống áo mẫu họa_tiết hàng gửi mur...,áo sơ_mi cotton dài tay trơn màu dành nam
